In [9]:
from PIL import ImageFont, ImageDraw, Image
import os
import numpy as np
import cv2 as cv
from collections import Iterator
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

N = 20

class MplColorHelper:

    def __init__(self, cmap_name, start_val, stop_val):
        if stop_val > 255:
            stop_val = 255
        self.cmap_name = cmap_name
        self.cmap = plt.get_cmap(cmap_name)
        self.norm = mpl.colors.Normalize(vmin=start_val, vmax=stop_val)
        self.scalarMap = cm.ScalarMappable(norm=self.norm, cmap=self.cmap)

    def get_rgb(self, val):
        rgb = list(self.scalarMap.to_rgba(val))[:3]
        ret = [int(i * 255) for i in rgb]
        #return rgb
        return [int(rgb[0]*255),int(rgb[1]*255),int(rgb[2]*255)]

COL = MplColorHelper('nipy_spectral', 0, 20)

class DataProvider(object):
    """
        
        path: string
            Path to directory were the files are stored.
    
    """
    
    def __init__(self,path,openIMG=True,pattern="scaled_17051.*"):
        def get_files(pwd,pattern=""):
    
            """

                Read all files from path which match a pattern and return a list with the names

            """

            import re
            files = []
            for file in os.listdir(pwd):
                matches = re.search(pattern, file)
                if matches:
                    files.append(matches[0])
            
            files.sort()
            return files

        self.files = get_files(path,pattern)
        self.path = path
        self.openIMG = openIMG
        
    def _openIMG(self,pwd):
        return np.array(Image.open(pwd))
        
    def __getitem__(self,i):
        if self.openIMG:
            return self._openIMG(os.path.join(self.path,self.files[i]))
        else:
            return os.path.join(self.path,self.files[i])
        
        
    def __len__(self):
        return len(self.files)
    
    def __iter__(self):
        self.n = -1
        return self
        
    def __next__(self):
        self.n += 1
        if self.n < len(self.files):
            if self.openIMG:
                return self._openIMG(os.path.join(self.path,self.files[self.n]))
            else:
                return os.path.join(self.path,self.files[self.n])
        else:
            raise StopIteration
    

class Cloud():
    def __init__(self,points,size = None):
        
        
        self.points = points
        if size is None:
            self.size = len(points[0])
        else:
            self.size = size
        self.color = [255,255,255]
        self.center_of_mass = int(np.sum(points[0])/len(points[0])),int(np.sum(points[1])/len(points[1]))
        self.min_x,self.max_x = points[0].min(),points[0].max()
        self.min_y,self.max_y = points[1].min(),points[1].max()  
            
    def __lt__(self, other):
        return self.size < other.size

    def __eq__(self, other):
        return self.size == other.size
    
    def __str__(self):
        return "Cloudsize: {}\nCenter : {}\n".format(self.size,self.center_of_mass)
    
    

    
    def addText(self,img,coord,text):

        font                   = cv.FONT_HERSHEY_COMPLEX_SMALL
        fontScale              = 0.5
        fontColor              = (255,255,255)
        lineType               = 1

        cv.putText(img,
                   text, 
                   coord,
                   font, 
                   fontScale,
                   fontColor,
                   lineType)


        return img

    
    def bounding_box(self,img):


        if self.size <= 1:
            return img
  
        ret = img
        ret[self.min_x:self.max_x,self.min_y] = 255
        ret[self.min_x:self.max_x,self.max_y] = 255
        ret[self.min_x,self.min_y:self.max_y] = 255
        ret[self.max_x,self.min_y:self.max_y] = 255
            


        ret = self.addText(ret,
                        (self.max_y,self.max_x+50),
                        "SWP: ("+str(self.center_of_mass[0])+","+str(self.center_of_mass[1])+")")
            
        ret[self.max_x:self.max_x+40,self.max_y] = 255

        return ret
    
    
    def draw(self,img):
        return self.bounding_box(img)
    
    def paintcolor(self,img):
        img[self.points] = self.color
        return img
        
    
    def dist(self,cloud):
        """
        
            returns euclidean dist between center of ma
        
        """
        
        return np.sqrt((cloud.center_of_mass[0] - self.center_of_mass[0])**2 +
                       (cloud.center_of_mass[1] - self.center_of_mass[1])**2)
        

    
    

class CloudTracker():
    
    """

        This class will label clouds via sequential labeling :
                
                
            max_dist: (int)
                Maximale Distanz so dass die Wolken als verbunden angesehen werden
            
            cloud_threshold: (int)
                Pixel die größer als dieser Threshold sind werden als Wolken klassifiziert
                                             
 
    """
        
    def __init__(self,data,max_dist=1,cloud_threshold=5):

        
        self.label = None
        self.max_dist = max_dist
        self.cloud_threshold = cloud_threshold
        self.data = data
        
        self.nbrs = list(np.arange(0,255))
        
        np.random.shuffle(self.nbrs)


    def max_contrast(self,img):
    
        """

            maximize contrast of images
            also deleting "edges"

        """

        img[img == img[0,0]] = 0

        mi,ma = img.min(),img.max()
        if ma == 0:
            return img

        img -= mi
        img[img == ma -mi] = 0
        ma = img.max()
        img = np.array(((img / ma) * 255),dtype='uint8')
        return img

    
    def binary(self,img,threshold=5):
        
        
        img[img > threshold ] = 255
        img[img <= threshold] = 0
        
        
        return img


    def sequentialLabeling(self,img):

        
        img[img >= 5] = 1
        x,y = np.where(img == 1)


        collision = dict()
        label = 2

        for i,j in zip(x,y):
            i_X = slice(i-self.max_dist,i+self.max_dist)
            j_Y = slice(j-self.max_dist,j+self.max_dist)

            window = img[i_X,j_Y]

            neighbours = np.argwhere(window > 1)


            if len(neighbours) == 0:
                window[window == 1] = label
                label +=1
                img[i_X,j_Y] = window

            elif len(neighbours) == 1:
                window[window == 1] = window[neighbours[0,0],neighbours[0,1]]
                img[i_X,j_Y] = window


            # handle label collisions

            else:
                k = np.amax(window)
                img[i,j] = k
                for index in neighbours:
                    nj = window[index[0], index[1]]

                    if nj != k:
                        if k not in collision:
                            collision[k] = set()
                        collision[k].add(nj)
                        if collision[k] is None:
                            del collision[k]



        def changeLabel(elem):
            c_label = collision[elem]
            for l in c_label:
                img[img == l] = elem


        def rearangeCollisions():
            for elem in collision:
                for item in collision[elem]:
                    if item in collision:
                        collision[elem] = (collision[elem] | collision[item])
                        collision[item] = set()
                if elem in collision[elem]:
                    collision[elem].remove(elem)

        rearangeCollisions()


        for i,elem in enumerate(collision):
            if collision[elem] is None:
                continue
            changeLabel(elem)

        cloud_size = []

        for i in range(2,label):
            a = len(np.where(img == i)[0])

            if a == 0:
                continue
            cloud_size.append((i,a))
        cloud_size = sorted(cloud_size, key=lambda x: x[1],reverse = True)

        return cloud_size



    def fit(self,img):
        pass
        
    
    def createClouds(self,labels,img,minSize = 60):
        """
        
            image has to be max_contrast + binary
        
        """
        colorindex = self.nbrs.copy()
        
        self.trackID += 1
        if len(self.cloudList) >= 2:
            self.cloudList.pop(0)
        
        clouds = []
        
        for l,size in labels:
            if size < minSize:
                continue
            indices = np.where(img == l)
            cloud = Cloud(indices)
            n = colorindex.pop(0)
            #print(n,N % n ,n % N)
            cloud.color = COL.get_rgb( n % N )
            #cloud.color=RGB_tuples[colorindex.pop(0) % N]
            
            clouds.append(cloud)
        
        sorted(clouds)

        for cloud in clouds:
            cloud.draw(img)
        self.cloudList.append(clouds)

    def find_Pair_Of_Clouds(self):
        if len(self.cloudList) < 2:
            return
        
        clouds_t0 = self.cloudList[0]
        clouds_t1 = self.cloudList[1]
        
        cloudDist = []
        for cloud_t0 in clouds_t0:

            dist = 10000
            nearest = None
            for cloud_t1 in clouds_t1:
                d = cloud_t0.dist(cloud_t1)
                if d <= dist:
                    dist = d
                    nearest = cloud_t1
            nearest.color = cloud_t0.color
            cloudDist.append( (cloud_t0,nearest) )

        return cloudDist
    
    def drawPairs(self,cloudDist):
        
        if cloudDist is None or len(self.imgList) < 2:
            return self.imgList[0]
        
        x,y = self.imgList[0].shape
        imgcon = np.zeros((x,2*y,3))
        img0 = cv.cvtColor(self.imgList[0],cv.COLOR_GRAY2RGB).copy()
        img1 = cv.cvtColor(self.imgList[1],cv.COLOR_GRAY2RGB).copy()

        
        
        for c0,c1 in cloudDist:
            img0 = c0.paintcolor(img0)
            img1 = c1.paintcolor(img1)
        imgcon[:,:y] = img0
        imgcon[:,y:] = img1
        imgcon[:x,y] = [255,255,255]
        return imgcon
        

    def create(self,inputPath, outputPath, delay, finalDelay, loop):
        cmd = "convert -delay {} {}*.png -delay {} -loop {} {}".format(
        delay, inputPath, finalDelay, loop,
        outputPath)
        print(cmd)
        os.system(cmd)

    
    def track(self,create_gif = False,name="clouds.gif",nbr_imgs = 0):
        
        self.trackID = 0
        self.cloudList = []
        self.imgList = []
        
        if create_gif: 
            folder = "GIF/"
            if not os.path.exists(folder):
                os.mkdir(folder)
        
        windowname = 'OpenCvFrame'
        cv.namedWindow(windowname)
        cv.moveWindow(windowname,2600,40)
        
        for i,pwd in enumerate(self.data):
            
            img = self.data._openIMG(pwd)
            file = pwd.split("/")[-1]

            img = self.max_contrast(img)
            img = self.binary(img)
            
            
            if len(self.imgList) >= 2:
                self.imgList.pop(0)
            self.imgList.append(img)
            label = self.sequentialLabeling(img)
            self.createClouds(label,img)
            cloudDist = self.find_Pair_Of_Clouds()
            

            
            img = self.drawPairs(cloudDist)

            cv.imshow(windowname, img)
            if create_gif and len(self.imgList) == 2:
            
                cv.imwrite(os.path.join(folder,file),img)
                if i == nbr_imgs and nbr_imgs != 0:
                    break
                
            if cv.waitKey(25) & 0XFF == ord('q'):
                ret = False
                break
        cv.destroyAllWindows()
        if create_gif:
            self.create(folder,name,20,250,0)



def tracking(file):
    pass

In [11]:
d = DataProvider("PNG",openIMG=False,pattern=".*")
labeling = CloudTracker(d,max_dist=15)
labeling.track(create_gif=True,name="track.gif",nbr_imgs=0)
cv.destroyAllWindows()

convert -delay 10 GIF/*.png -delay 250 -loop 0 track.gif
